### Using XGBoost in Python
First of all, just like what you do with any other dataset, you are going to import the dataset and store it in a variable called case8.

In [3]:
import pandas as pd

case8 = pd.read_csv (r'C:\Users\04616598\Box Sync\Documents\Data Sciance\msds_7333QuantifyingTheWorld\MSDS7333\CaseStudy08\data\case_8.csv')

case8.head()

,ID,target,v1,v2,v3,v4,v5,v6,v7,v8,...,v122,v123,v124,v125,v126,v127,v128,v129,v130,v131
0,3,1,1.335739,8.727474,C,3.921026,7.915266,2.599278,3.176895,0.012941,...,8.000000,1.989780,0.035754,AU,1.804126,3.113719,2.024285,0,0.636365,2.857144
1,4,1,1.630686,7.464411,C,4.145098,9.191265,2.436402,2.483921,2.301630,...,6.822439,3.549938,0.598896,AF,1.672658,3.239542,1.957825,0,1.925763,1.739389
2,5,1,0.943877,5.310079,C,4.410969,5.326159,3.979592,3.928571,0.019645,...,9.333333,2.477596,0.013452,AE,1.773709,3.922193,1.120468,2,0.883118,1.176472
3,6,1,0.797415,8.304757,C,4.225930,11.627438,2.097700,1.987549,0.171947,...,7.018256,1.812795,0.002267,CJ,1.415230,2.954381,1.990847,1,1.677108,1.034483
4,8,1,1.630686,7.464411,C,4.145098,8.742359,2.436402,2.483921,1.496569,...,6.822439,3.549938,0.919812,Z,1.672658,3.239542,2.030373,0,1.925763,1.739389


One Hot encoding of categorical features.

In [4]:
case8 = pd.get_dummies(case8)

In [5]:
# use pd.concat to join the new columns with your original dataframe
#case8 = pd.concat([case8,pd.get_dummies(case8['v3'], prefix='v3')],axis=1)

# now drop the original 'country' column (you don't need it anymore)
#case8.drop(['v3'],axis=1, inplace=True)

You can check for its keys using the .keys() method.

In [6]:
print (case8.keys())

Index(['ID', 'target', 'v1', 'v2', 'v4', 'v5', 'v6', 'v7', 'v8', 'v9',
       ...
       'v125_Q', 'v125_R', 'v125_S', 'v125_T', 'v125_U', 'v125_V', 'v125_W',
       'v125_X', 'v125_Y', 'v125_Z'],
      dtype='object', length=18688)


You can easily check for its shape by using the case8.shape attribute, which will return the size of the dataset.

In [7]:
print (case8.shape)

(114321, 18688)


As you can see it returned (114321, 18688), that means there are 114321 rows of data with 18688 columns.

Run the .info() method on your DataFrame to get useful information about the data.

In [8]:
case8.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 114321 entries, 0 to 114320
Columns: 18688 entries, ID to v125_Z
dtypes: float64(108), int64(6), uint8(18574)
memory usage: 2.1 GB


To get more summary statistics of the different features in the dataset we use the describe() method on the case8 DataFrame.

**Note** - describe() only gives summary statistics of columns which are continuous in nature and not categorical.

In [9]:
#case8.describe()

,ID,target,v1,v2,v4,v5,v6,v7,v8,v9,...,v125_Q,v125_R,v125_S,v125_T,v125_U,v125_V,v125_W,v125_X,v125_Y,v125_Z
count,114321.000000,114321.000000,1.143210e+05,1.143210e+05,1.143210e+05,1.143210e+05,1.143210e+05,1.143210e+05,1.143210e+05,1.143210e+05,...,114321.000000,114321.000000,114321.000000,114321.000000,114321.000000,114321.000000,114321.000000,114321.000000,114321.000000,114321.000000
mean,114228.928228,0.761199,1.630686e+00,7.464411e+00,4.145098e+00,8.742359e+00,2.436402e+00,2.483921e+00,1.496569e+00,9.031859e+00,...,0.009325,0.010645,0.007077,0.005380,0.009473,0.028289,0.007768,0.007584,0.007698,0.013952
std,65934.487362,0.426353,8.132649e-01,2.225036e+00,8.626621e-01,1.543441e+00,4.506138e-01,4.427150e-01,2.109786e+00,1.449542e+00,...,0.096113,0.102627,0.083825,0.073148,0.096869,0.165797,0.087791,0.086755,0.087398,0.117292
min,3.000000,0.000000,-9.996497e-07,-9.817614e-07,-6.475929e-07,-5.287068e-07,-9.055091e-07,-9.468765e-07,-7.783778e-07,-9.828757e-07,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
25%,57280.000000,1.000000,1.346153e+00,6.575770e+00,4.068697e+00,8.394090e+00,2.340968e+00,2.376586e+00,2.653147e-01,8.813560e+00,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
50%,114189.000000,1.000000,1.630686e+00,7.464411e+00,4.145098e+00,8.742359e+00,2.436402e+00,2.483921e+00,1.496569e+00,9.031859e+00,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
75%,171206.000000,1.000000,1.630686e+00,7.551501e+00,4.340229e+00,8.924798e+00,2.484699e+00,2.528445e+00,1.496569e+00,9.302325e+00,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
max,228713.000000,1.000000,2.000000e+01,2.000000e+01,2.000000e+01,2.000000e+01,2.000000e+01,2.000000e+01,2.000000e+01,2.000000e+01,...,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000


We will build the model using Trees as base learners (which are the default base learners) using XGBoost's scikit-learn compatible API. Along the way, we will also use some of the common tuning parameters which XGBoost provides in order to improve our model's performance, and using the root mean squared error (RMSE) performance metric to check the performance of the trained model on the test set. Root mean Squared error is the square root of the mean of the squared differences between the actual and the predicted values. We start by importing the library xgboost and other important libraries that we will be using for building the model.

In [10]:
import xgboost as xgb
import numpy as np
from sklearn.metrics import mean_squared_error

Create a sparce matrix

In [11]:
# import sparse module from SciPy package 
from scipy import sparse
# import uniform module to create random numbers
from scipy.stats import uniform

In [2]:
case8_spr = sparse.csr_matrix(case8)

NameError: name 'case8' is not defined

Separate the target variable and rest of the variables using .iloc to subset the data.

In [9]:
X, y = case8.iloc[:,:-1],case8.iloc[:,-1]

In [12]:
data_dmatrix = xgb.DMatrix(data=X,label=y)

MemoryError: Unable to allocate 15.9 GiB for an array with shape (114321, 18687) and data type float64

Now, we will create the train and test set for cross-validation of the results using the train_test_split function from sklearn's model_selection module with test_size size equal to 20% of the data. Also, to maintain reproducibility of the results, a random_state is also assigned.

In [11]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=123)